# Complete Visualization Demo: Classifier Model Interpreter

This notebook demonstrates all visualization capabilities including **NEW FEATURES**:
- **Threshold Detection**: Find where feature effects change significantly
- **Auto Segment Discovery**: Find behavioral groups based on SHAP patterns
- **Local Explanations**: Waterfall/Force plots for individual predictions

## What Makes This Package Better Than Native SHAP?

### Key Advantages:

1. **Interactive Plotly visualizations** instead of static matplotlib
2. **Prediction surface visualizations** (contour & 3D) - NOT in SHAP
3. **Threshold detection** - finds non-linear breakpoints automatically
4. **Segment discovery** - finds behavioral groups traditional analytics misses
5. **Local explanation plots** - waterfall charts for individual predictions
6. **Simple, integrated API** - one Interpreter class for everything

## Setup

In [1]:
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from src.core import Interpreter
import warnings
warnings.filterwarnings('ignore')

print("Setup complete")

Setup complete


In [2]:
# Load the NEW realistic customer conversion data
data_path = Path.cwd().parent / 'data' / 'customer_conversion.csv'
df = pd.read_csv(data_path)

print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Conversion rate: {df['converted'].mean():.1%}")
print(f"\nFeatures: {list(df.columns)}")

Dataset: 5000 rows, 19 columns
Conversion rate: 72.9%

Features: ['customer_id', 'converted', 'days_since_login', 'engagement_score', 'time_on_site_mins', 'pages_viewed', 'loyalty_points', 'account_age_months', 'previous_purchases', 'cart_value', 'competitor_price_diff', 'discount_pct', 'email_opens_last_30d', 'session_count_30d', 'customer_tier', 'channel', 'device_type', 'region', 'referral_source']


In [3]:
# Prepare data - label encode categoricals
cat_cols = ['customer_tier', 'channel', 'device_type', 'region', 'referral_source']
X = df.drop(['customer_id', 'converted'], axis=1).copy()
y = df['converted'].values

label_encoders = {}
for col in cat_cols:
    if col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        label_encoders[col] = le

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)
model.fit(X_train, y_train)

print(f"Test accuracy: {model.score(X_test, y_test):.3f}")

Test accuracy: 0.729


In [4]:
# Initialize Interpreter
interp = Interpreter(model, X_test, y_test, config='detailed_analysis')

# Create label mappings for categorical visualizations
tier_labels = {i: label for i, label in enumerate(label_encoders['customer_tier'].classes_)}
channel_labels = {i: label for i, label in enumerate(label_encoders['channel'].classes_)}
device_labels = {i: label for i, label in enumerate(label_encoders['device_type'].classes_)}

print("Interpreter ready")
print(f"\nLabel mappings:")
print(f"  Customer Tier: {tier_labels}")
print(f"  Channel: {channel_labels}")

Interpreter ready

Label mappings:
  Customer Tier: {0: 'bronze', 1: 'gold', 2: 'platinum', 3: 'silver'}
  Channel: {0: 'email', 1: 'mobile_app', 2: 'social', 3: 'web'}


---
## 1. Global Feature Importance

**What it shows:** Which features matter most overall

In [5]:
fig = interp.plot_global_importance(top_n=15)
fig.show()

---
## 2. Beeswarm Plot

**What it shows:** Distribution of SHAP values for each feature

In [6]:
fig = interp.plot_beeswarm(top_n=12)
fig.show()

---
## 3. Feature Dependence Plots

**What it shows:** How feature values affect predictions

In [7]:
fig = interp.plot_dependence('days_since_login')
fig.show()

In [8]:
fig = interp.plot_dependence('discount_pct')
fig.show()

In [9]:
fig = interp.plot_dependence('loyalty_points')
fig.show()

---
## 4. Categorical Dependence Plots

**What it shows:** SHAP distributions for each category (box plots)

In [10]:
fig = interp.plot_dependence_categorical('customer_tier', value_labels=tier_labels)
fig.show()

In [11]:
fig = interp.plot_dependence_categorical('channel', value_labels=channel_labels)
fig.show()

---
# NEW FEATURE: Threshold Detection

## 5. Automatic Threshold Detection

**What it does:** Finds values where feature effects CHANGE significantly

**Why it matters:** Traditional analytics shows averages. This finds specific breakpoints:
- "Effect flips at 14 days"
- "Discount only matters above 15%"

**NATIVE SHAP DOESN'T HAVE THIS!**

In [12]:
# Detect thresholds for days_since_login
threshold_result = interp.detect_thresholds('days_since_login')

print("="*70)
print("THRESHOLD DETECTION: days_since_login")
print("="*70)
print(f"\nMethod: {threshold_result['method']}")
print(f"Samples analyzed: {threshold_result['n_samples']}")

print("\nDetected Thresholds:")
print("-"*70)
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at: {t['value']:.0f} days")
    print(f"    SHAP before: {t['shap_before']:.4f}")
    print(f"    SHAP after:  {t['shap_after']:.4f}")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    Confidence: {t['confidence']:.1%}")
    print(f"    -> {t['interpretation']}")

THRESHOLD DETECTION: days_since_login

Method: changepoint
Samples analyzed: 1000

Detected Thresholds:
----------------------------------------------------------------------

  Threshold at: 47 days
    SHAP before: 0.5324
    SHAP after:  -1.4753
    Effect change: -2.0077
    Confidence: 100.0%
    -> Effect dramatically decreases at days_since_login=47.00: from 0.532 to -1.475

  Threshold at: 15 days
    SHAP before: 1.2729
    SHAP after:  -0.6841
    Effect change: -1.9570
    Confidence: 100.0%
    -> Effect dramatically decreases at days_since_login=15.00: from 1.273 to -0.684

  Threshold at: 59 days
    SHAP before: 0.4173
    SHAP after:  -1.4256
    Effect change: -1.8429
    Confidence: 100.0%
    -> Effect dramatically decreases at days_since_login=59.00: from 0.417 to -1.426


In [13]:
# Detect thresholds for discount_pct
threshold_result = interp.detect_thresholds('discount_pct')

print("="*70)
print("THRESHOLD DETECTION: discount_pct")
print("="*70)

print("\nDetected Thresholds:")
for t in threshold_result['thresholds']:
    print(f"\n  Threshold at: {t['value']:.0f}%")
    print(f"    Effect change: {t['effect_change']:+.4f}")
    print(f"    -> {t['interpretation']}")

THRESHOLD DETECTION: discount_pct

Detected Thresholds:

  Threshold at: 25%
    Effect change: +1.1655
    -> Effect dramatically increases at discount_pct=25.00: from -0.079 to 1.087

  Threshold at: 30%
    Effect change: +1.1620
    -> Effect dramatically increases at discount_pct=30.00: from 0.017 to 1.179

  Threshold at: 20%
    Effect change: +1.1482
    -> Effect dramatically increases at discount_pct=20.00: from -0.215 to 0.933


In [14]:
# Detect thresholds across ALL top features
all_thresholds = interp.detect_all_thresholds(top_n_features=8)

print("ALL DETECTED THRESHOLDS (Top 8 Features)")
print("="*70)
all_thresholds[['feature', 'value', 'effect_change', 'interpretation']].head(10)

ALL DETECTED THRESHOLDS (Top 8 Features)


,feature,value,effect_change,interpretation
0,days_since_login,47.00,-2.007722,Effect dramatically decreases at days_since_lo...
1,days_since_login,15.00,-1.956963,Effect dramatically decreases at days_since_lo...
2,days_since_login,59.00,-1.842903,Effect dramatically decreases at days_since_lo...
3,discount_pct,25.00,1.165516,Effect dramatically increases at discount_pct=...
4,discount_pct,30.00,1.161969,Effect dramatically increases at discount_pct=...
5,discount_pct,20.00,1.148203,Effect dramatically increases at discount_pct=...
9,account_age_months,38.00,0.736913,Effect dramatically increases at account_age_m...
6,competitor_price_diff,20.83,-0.644753,Effect dramatically decreases at competitor_pr...
10,account_age_months,50.00,0.621930,Effect dramatically increases at account_age_m...
11,account_age_months,25.00,0.590745,Effect dramatically increases at account_age_m...


---
# NEW FEATURE: Segment Discovery

## 6. Auto Segment Discovery

**What it does:** Finds groups of customers where the MODEL REASONS DIFFERENTLY

**Why it's different from traditional segmentation:**
- Traditional: Groups by demographics (age, tier)
- This: Groups by SHAP patterns (how model explains predictions)

Reveals behavioral segments like:
- "Loyalty Driven" - predictions driven by loyalty points
- "Price Sensitive" - predictions driven by discounts
- "Engagement Dependent" - predictions driven by recency

**NATIVE SHAP DOESN'T HAVE THIS!**

In [15]:
# Discover behavioral segments
segments = interp.discover_segments(n_segments=4, top_n_features=10)

# Print summary
print(interp.get_segment_summary(segments))

DISCOVERED SEGMENTS SUMMARY

Method: kmeans
Number of segments: 4
Features used for clustering: days_since_login, discount_pct, competitor_price_diff, account_age_months, cart_value...

----------------------------------------------------------------------
SEGMENT 0: Days Since Login Driven (strongly dominated by days_since_login)
----------------------------------------------------------------------
Size: 280 observations (28.0% of total)
Target rate: 57.5%

Top Drivers:
  days_since_login               -0.8919
  discount_pct                   +0.4120
  competitor_price_diff          -0.2951
  cart_value                     +0.2017
  channel                        -0.1744

----------------------------------------------------------------------
SEGMENT 1: Days Since Login Driven (with account_age_months)
----------------------------------------------------------------------
Size: 135 observations (13.5% of total)
Target rate: 79.3%

Top Drivers:
  days_since_login               +1.1447


In [16]:
# Visualize segment profiles
fig = interp.plot_segment_profiles(segments, top_n_features=8)
fig.show()

In [17]:
# Compare feature importance across segments
fig = interp.plot_segment_comparison(segments, top_n=10)
fig.show()

---
# NEW FEATURE: Local Explanations

## 7. Waterfall & Force Plots

**What it shows:** How a SINGLE prediction was made

- Base value = average model output
- Each feature pushes prediction up or down
- Final prediction = sum of all contributions

Essential for:
- Explaining individual decisions
- Understanding outliers
- Debugging predictions

In [18]:
# Find a high-probability conversion and a low-probability one
y_proba = model.predict_proba(X_test)[:, 1]

high_prob_idx = np.argmax(y_proba)  # Highest conversion probability
low_prob_idx = np.argmin(y_proba)   # Lowest conversion probability
mid_idx = np.argsort(y_proba)[len(y_proba)//2]  # Middle

print(f"High probability observation: idx={high_prob_idx}, prob={y_proba[high_prob_idx]:.3f}")
print(f"Low probability observation: idx={low_prob_idx}, prob={y_proba[low_prob_idx]:.3f}")
print(f"Mid probability observation: idx={mid_idx}, prob={y_proba[mid_idx]:.3f}")

High probability observation: idx=313, prob=0.999
Low probability observation: idx=718, prob=0.029
Mid probability observation: idx=45, prob=0.856


In [19]:
# Waterfall plot for HIGH probability prediction
fig = interp.plot_waterfall(high_prob_idx, top_n=10)
fig.show()

print("\nThis customer has HIGH conversion probability because:")
print(interp.explain_observation_text(high_prob_idx, top_n=5))


This customer has HIGH conversion probability because:
Observation #313 Explanation
Base prediction (average): 1.030
Final prediction: 6.878
Total SHAP adjustment: +5.848

Top Contributing Features:
--------------------------------------------------
  days_since_login = 10.0  (+1.2910, increases prediction)
  discount_pct = 30.0  (+0.9826, increases prediction)
  account_age_months = 58.0  (+0.8563, increases prediction)
  time_on_site_mins = 0.5  (+0.6609, increases prediction)
  cart_value = 53.64  (+0.3424, increases prediction)


In [20]:
# Waterfall plot for LOW probability prediction
fig = interp.plot_waterfall(low_prob_idx, top_n=10)
fig.show()

print("\nThis customer has LOW conversion probability because:")
print(interp.explain_observation_text(low_prob_idx, top_n=5))


This customer has LOW conversion probability because:
Observation #718 Explanation
Base prediction (average): 1.030
Final prediction: -3.516
Total SHAP adjustment: -4.545

Top Contributing Features:
--------------------------------------------------
  days_since_login = 40.0  (-0.7805, decreases prediction)
  engagement_score = 69.6  (-0.6733, decreases prediction)
  session_count_30d = 1.0  (-0.6544, decreases prediction)
  loyalty_points = 50.0  (-0.5453, decreases prediction)
  competitor_price_diff = 11.9  (-0.3466, decreases prediction)


In [21]:
# Force plot (alternative horizontal view)
fig = interp.plot_force(high_prob_idx, top_n=12)
fig.show()

In [22]:
# Compare multiple observations side by side
fig = interp.plot_multiple_observations([high_prob_idx, mid_idx, low_prob_idx], top_n=8)
fig.show()

---
## 8. Interaction Detection

**What it shows:** Which feature pairs interact most

In [23]:
interactions = interp.detect_interactions(top_n=15, method='shap_variance')
print("Top 15 Feature Interactions:")
interactions

Top 15 Feature Interactions:


,feature_1,feature_2,interaction_strength
5,days_since_login,previous_purchases,0.186636
9,days_since_login,email_opens_last_30d,0.162772
8,days_since_login,discount_pct,0.143044
4,days_since_login,account_age_months,0.141130
13,days_since_login,device_type,0.138778
3,days_since_login,loyalty_points,0.103230
15,days_since_login,referral_source,0.092339
10,days_since_login,session_count_30d,0.076022
12,days_since_login,channel,0.075789
108,discount_pct,email_opens_last_30d,0.064364


---
## 9. Prediction Surface Visualizations

**What it shows:** Predicted probability across two features

**Unique to this package** - shows actual predictions (Y), not SHAP values

In [24]:
# Blocky heatmap: discount × customer tier
fig = interp.plot_interaction_contour(
    'discount_pct', 
    'customer_tier',
    value_labels_2=tier_labels
)
fig.show()

print("\nBusiness insight: How does discount effectiveness vary by customer tier?")


Business insight: How does discount effectiveness vary by customer tier?


In [25]:
# Continuous features
fig = interp.plot_interaction_contour('days_since_login', 'engagement_score', n_grid=40)
fig.show()

In [26]:
# 3D surface plot
fig = interp.plot_interaction_surface_3d('discount_pct', 'loyalty_points', n_grid=25)
fig.show()

---
## 10. Conditional Dependence Plots

**What it shows:** How a feature's effect varies by another feature

- Parallel lines = no interaction
- Diverging lines = strong interaction

In [27]:
# Does discount effect vary by customer tier?
fig = interp.plot_conditional_dependence(
    'discount_pct', 
    'customer_tier',
    value_labels=tier_labels
)
fig.show()

print("Business question: Does discount effectiveness vary by customer tier?")
print("If lines diverge -> YES, target different tiers with different discounts")

Business question: Does discount effectiveness vary by customer tier?
If lines diverge -> YES, target different tiers with different discounts


In [28]:
# Does engagement effect vary by channel?
fig = interp.plot_conditional_dependence(
    'engagement_score', 
    'channel',
    value_labels=channel_labels
)
fig.show()

---
## 11. Model Performance

**Includes:** Confusion matrix, ROC curve, metrics summary, calibration curve

In [29]:
performance = interp.plot_performance()

performance['metrics_summary'].show()

In [30]:
performance['confusion_matrix'].show()

In [31]:
performance['roc_curve'].show()

---
# Summary: New Features in This Package

## NEW Features (Not in SHAP)

1. **Threshold Detection** - Finds non-linear breakpoints automatically
   - "Effect changes at 14 days"
   - "Discount matters above 15%"

2. **Segment Discovery** - Finds behavioral groups based on SHAP patterns
   - Groups by HOW model reasons, not demographics
   - Reveals hidden customer types

3. **Local Explanations** - Waterfall/Force plots for individual predictions
   - Explain specific decisions
   - Compare high vs low predictions

4. **Prediction Surfaces** - 2D/3D visualizations of actual predictions
   - Shows probabilities, not abstract SHAP values
   - Business-friendly interpretation

5. **Conditional Dependence** - Shows interactions visually
   - Parallel lines = no interaction
   - Diverging lines = interaction exists

## Why This Package Beats Native SHAP

- **Interactive Plotly** vs static matplotlib
- **Business-focused** - shows predictions, not just SHAP
- **Discovers insights** - thresholds, segments, interactions
- **Simple API** - one Interpreter class
- **Great for presentations** - interactive, professional visuals